In [ ]:
from __future__ import annotations
import ee

""" Class used to store information about points and operations on them """


class PointData:
    def __init__(self, latitude: float, longitude: float, projection: ee.Projection, gee_point: ee.Geometry.Point = None):
        self.__coordinates_degrees = (latitude, longitude)
        self.__gee_point = gee_point if gee_point is not None else ee.Geometry.Point(latitude, longitude)
        self.__coordinates_meters = None
        self.__projection = projection

    @classmethod
    def from_gee_point(cls, gee_point: ee.Geometry.Point, projection: ee.Projection):
        """ Construction of class object using Google Earth Engine Point """
        latitude, longitude = gee_point.coordinates().getInfo()
        return cls(latitude, longitude, projection, gee_point)

    @classmethod
    def from_coordinates_meters(cls, x: float, y: float, projection: ee.Projection):
        """ Construction of class object using coordinates in meters """
        point = ee.Geometry.Point([x, y], projection).transform("EPSG:4326")
        created_point = PointData.from_gee_point(point, projection)
        created_point.__coordinates_meters = x, y
        return created_point

    def get_gee_point(self):
        """ Returns representation of point on Google Earth Engine server side """
        return self.__gee_point

    def get_coordinates_degrees(self):
        """ Returns coordinates of point in degrees """
        return self.__coordinates_degrees

    def get_coordinates_meters(self):
        """ Returns coordinates of point in meters """
        if self.__coordinates_meters is None:
            self.__coordinates_meters = self.__gee_point.transform(self.__projection).coordinates().getInfo()
        return self.__coordinates_meters

In [ ]:
import ee
import geemap
#from PointData import PointData

"""Class detecting edges in images using different bands from Sentinel2"""


class EdgeDetector:
    def __init__(self, points: ee.FeatureCollection, map_center: PointData):
        self.__time_periods = [['2023-06-01', '2023-10-01'], ['2024-06-01', '2024-10-01']]  # more time periods can provide more
        # accurate data, but also makes calculations slower
        self.__bands = ['B4', 'B8']  # bands of satellite imagery that are used for edge detection
        self.__thresholds = [150, 160, 150, 160]  # low and high threshold that is used for each band [low_band1, high_band1, ... ]
        self.__sigmas = [7, 8.5]  # values of sigma that are used for each of the bands
        self.__distance = 5  # max distance between edges to be connected
        self.__scale = 16  # scale that is used to show results on GEE Map
        self.__points = points
        self.__map_center = map_center
        self.__cloud_filter_threshold = 5

    # Finding selected band in a selected place
    def __get_band_image(self, band: str, time_period: list) -> ee.Image:
        """ Function that finds particular band for selected image """
        return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                .filterDate(time_period[0], time_period[1])
                .filterBounds(self.__map_center.get_gee_point())
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', self.__cloud_filter_threshold))
                .select(band)
                .first())

    # Clouds masking is used only for comparison and visualization
    # TODO: change this function
    def __mask_s2_clouds(self, image: ee.Image) -> ee.Image:
        """Masks clouds in a Sentinel-2 image using the QA band.

        Args:
            image (ee.Image): A Sentinel-2 image.

        Returns:
            ee.Image: A cloud-masked Sentinel-2 image.
        """
        qa = image.select('QA60')

        # Bits 10 and 11 are clouds and cirrus, respectively.
        cloud_bit_mask = 1 << 10
        cirrus_bit_mask = 1 << 11

        # Both flags should be set to zero, indicating clear conditions.
        mask = (
            qa.bitwiseAnd(cloud_bit_mask)
            .eq(0)
            .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        )

        return image.updateMask(mask).divide(10000)

    def __get_RGB_map(self) -> ee.Image:
        """ Function that returns map image for a GEE Map """
        return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                .filterDate(self.__time_periods[0][0], self.__time_periods[-1][0])
                .filterBounds(self.__map_center.get_gee_point())
                # Pre-filter to get less cloudy granules.
                .filter(
            ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', self.__cloud_filter_threshold)
        )
                .map(self.__mask_s2_clouds)
                .first())

    def __run_canny_for_bands(self, bands_data: list) -> ee.Image:
        """ Function runs Canny edge detection algorithm for every provided band """
        bands_after_canny = []
        for i in range(len(bands_data)):
            low_threshold_detection = (
                ee.Algorithms.CannyEdgeDetector(
                    image=bands_data[i], threshold=self.__thresholds[i], sigma=self.__sigmas[i]
                )
            ).gt(0)
            high_threshold_detection = (
                ee.Algorithms.CannyEdgeDetector(
                    image=bands_data[i], threshold=self.__thresholds[i + 1], sigma=self.__sigmas[i]
                )
            ).gt(0)
            hysteresis_threshold = low_threshold_detection.And(
                high_threshold_detection.focal_max(self.__distance, 'square', 'pixels'))
            bands_after_canny.append(high_threshold_detection.Or(hysteresis_threshold))

        if len(bands_after_canny) > 1:
            aggregated_canny = bands_after_canny[0].Or(bands_after_canny[1])
            for i in range(2, len(bands_after_canny)):
                aggregated_canny = aggregated_canny.Or(bands_after_canny[i])
            return aggregated_canny.select([aggregated_canny.bandNames().get(0)])
        return bands_after_canny[0]

    def __prepare_result_map(self, results_list: ee.Image) -> geemap.Map:
        """ Prepares result map that is ready to show to user """
        resultMap = geemap.Map()
        center_coords = self.__map_center.get_coordinates_degrees()
        resultMap.set_center(center_coords[0], center_coords[1], self.__scale)
        visualization = {
            'min': 0.0,
            'max': 0.3,
            'bands': ['B4', 'B3', 'B2'],
        }
        resultMap.addLayer(self.__get_RGB_map(), visualization, "Source map")
        for i in range(len(results_list)):
            resultMap.addLayer(results_list[i].updateMask(results_list[i]), {"palette": ["ffffff"]},
                               self.__time_periods[i][0] + " - " + self.__time_periods[i][1])
        resultMap.addLayer(self.__points, {'color': 'red'}, 'User input points')
        return resultMap

    def __detect_edges(self) -> list[ee.Image]:
        """ Detects edged using Canny detector for selected periods """
        aggregated_canny_results = []
        for time_period in self.__time_periods:
            bands_data = []
            for band in self.__bands:
                bands_data.append(self.__get_band_image(band, time_period))
            aggregated_canny_results.append(self.__run_canny_for_bands(bands_data))
        return aggregated_canny_results

    def detect_and_show_on_map(self) -> geemap.Map:
        """ You can use this function to show results on GEE Map """
        aggregated_canny_results = self.__detect_edges()

        return self.__prepare_result_map(aggregated_canny_results)

    def detect_and_return_merged_bands(self) -> geemap.Map:
        """ Returns GEE Map that containes merged bands for all periods into one image """
        aggregated_canny_results = self.__detect_edges()

        # Merging all the detected edges into one band
        results = ee.ImageCollection(aggregated_canny_results).toBands()
        return results.reduce(ee.Reducer.max())


NameError: name 'PointData' is not defined

In [ ]:
import ee
#from AreaDetector import AreaDetector
#from EdgeDetector import EdgeDetector
#from PointData import PointData

""" Class that controls detection of areas and returns results of it as a polygon points """


class AreaDetectionController:

    def __init__(self, points_list: list[PointData]):
        self.__area_detector = None
        self.__projection = ee.Projection('EPSG:3035')  # projection that is accurate only to Europe!
        self.__points_list = points_list
        self.__points_feature_collection = ee.FeatureCollection(
            [ee.Feature(point.get_gee_point().transform(self.__projection)) for point in
             points_list])  # point is created as EPSG:4326 projection by default
        self.__map_center = PointData.from_gee_point(self.__points_feature_collection.geometry().centroid(), self.__projection)
        self.__edge_detector = EdgeDetector(self.__points_feature_collection, self.__map_center)

    def run_area_detection(self):
        """ Detects area for all points that were provided to class """
        print("Detection of edges is starting")
        detected_edges_map = self.__edge_detector.detect_and_show_on_map()
        print("Detection of edges is finished")
        return detected_edges_map

        # self.__area_detector = AreaDetector(detected_edges_map, self.__map_center, self.__projection)
        #
        # print("Detection of areas is starting")
        # self.__area_detector.detect_areas(self.__points_list)
        # print("Detection of areas is finished")
        #
        # print("Point extraction is starting")
        # self.__area_detector.prepare_for_points_extraction()
        # self.__area_detector.get_boundary_points()  # this function returns points for path planning
        # print("Point extraction is finished")
        #
        # self.__area_detector.plot_result(self.__points_list)


In [ ]:
import ee
import time
#from AreaDetectionController import AreaDetectionController
#from PointData import PointData

start = time.time()
ee.Authenticate()
ee.Initialize(project="uav-route-planning")

# code to execute
""" Detection using multiple points """

longitude, latitude = 53.324389, 18.455298
point = PointData(latitude, longitude, ee.Projection('EPSG:3035'))
points_list = [point]
areaDetectionController = AreaDetectionController(points_list)
map = areaDetectionController.run_area_detection()

print("Total time:", str(time.time() - start), "seconds")
map
# dla b4 (czerwony), rozmycie 7, progowanie 150-160

In [ ]:
import ee
import time
#from AreaDetectionController import AreaDetectionController
#from PointData import PointData

start = time.time()
ee.Authenticate()
ee.Initialize(project="uav-route-planning")

# code to execute
""" Detection using multiple points """

longitude, latitude = 53.324389, 18.455298
point = PointData(latitude, longitude, ee.Projection('EPSG:3035'))
points_list = [point]
areaDetectionController = AreaDetectionController(points_list)
map = areaDetectionController.run_area_detection()

print("Total time:", str(time.time() - start), "seconds")
map
# b5 (Red Edge 1), rozmycie 7, threshold 140-150

Detection of edges is starting
Detection of edges is finished
Total time: 3.7169065475463867 seconds


Map(center=[53.324389075449005, 18.455297999999992], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
import ee
import time
#from AreaDetectionController import AreaDetectionController
#from PointData import PointData

start = time.time()
ee.Authenticate()
ee.Initialize(project="uav-route-planning")

# code to execute
""" Detection using multiple points """

longitude, latitude = 53.324389, 18.455298
point = PointData(latitude, longitude, ee.Projection('EPSG:3035'))
points_list = [point]
areaDetectionController = AreaDetectionController(points_list)
map = areaDetectionController.run_area_detection()

print("Total time:", str(time.time() - start), "seconds")
map
# b7 (Red Edge 3), rozmycie 6, threshold 140-150

Detection of edges is starting
Detection of edges is finished
Total time: 4.430203676223755 seconds


Map(center=[53.324389075449005, 18.455297999999992], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
import ee
import time
#from AreaDetectionController import AreaDetectionController
#from PointData import PointData

start = time.time()
ee.Authenticate()
ee.Initialize(project="uav-route-planning")

# code to execute
""" Detection using multiple points """

longitude, latitude = 53.324389, 18.455298
point = PointData(latitude, longitude, ee.Projection('EPSG:3035'))
points_list = [point]
areaDetectionController = AreaDetectionController(points_list)
map = areaDetectionController.run_area_detection()

print("Total time:", str(time.time() - start), "seconds")
map
# b8 (NIR), rozmycie 8.5, threshold 150-160 nie umiem tego dostroić chyba

Detection of edges is starting
Detection of edges is finished
Total time: 4.279369592666626 seconds


Map(center=[53.324389075449005, 18.455297999999992], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
import ee
import time
#from AreaDetectionController import AreaDetectionController
#from PointData import PointData

start = time.time()
ee.Authenticate()
ee.Initialize(project="uav-route-planning")

# code to execute
""" Detection using multiple points """

longitude, latitude = 53.324389, 18.455298
point = PointData(latitude, longitude, ee.Projection('EPSG:3035'))
points_list = [point]
areaDetectionController = AreaDetectionController(points_list)
map = areaDetectionController.run_area_detection()

print("Total time:", str(time.time() - start), "seconds")
map
# b11 (SWIR 1), rozmycie 7, threshold 200-210

Detection of edges is starting
Detection of edges is finished
Total time: 4.166543960571289 seconds


Map(center=[53.324389075449005, 18.455297999999992], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# bierzemy NIR, czerwony i swir
# b11 (SWIR 1), rozmycie 7, threshold 200-210
# b8 (NIR), rozmycie 8.5, threshold 150-160
# dla b4 (czerwony), rozmycie 7, progowanie 150-160
import ee
import time
#from AreaDetectionController import AreaDetectionController
#from PointData import PointData

start = time.time()
ee.Authenticate()
ee.Initialize(project="uav-route-planning")

# code to execute
""" Detection using multiple points """

longitude, latitude = 53.324389, 18.455298
point = PointData(latitude, longitude, ee.Projection('EPSG:3035'))
points_list = [point]
areaDetectionController = AreaDetectionController(points_list)
map = areaDetectionController.run_area_detection()

print("Total time:", str(time.time() - start), "seconds")
map

Detection of edges is starting
Detection of edges is finished
Total time: 4.074002742767334 seconds


Map(center=[53.324389075449005, 18.455297999999992], controls=(WidgetControl(options=['position', 'transparent…